# Lab 4
### Introduction to Machine Learning, 2021-2022 period 4

This assignment is to be done with a partner. Please only submit ONE .ipynb (not .py) file per pair!

There are **11** problems in this assignment

**Total points: 10**

**Deadline: 2022-06-17 17:00**

**Names and student ids: Andreas Meeldijk 0892734 - Mote van Reij 0990345**

---

# Introduction

In this laboratory we will experiment with text classification. The inputs to our machine learning models will be texts, each of which has a label (0 or 1). The model is trained to classify new texts as 0 or 1. In particular, we will use the dataset from the shared task *Presupposed Taxonomies: Evaluating Neural Network Semantics (PreTENS)*, part of the SemEval 2022 conference:

https://sites.google.com/view/semeval2022-pretens

The data consists of simple sentences, such as these:

    * I like trees, and in particular birches
    * I like oaks, and in particular trees
    
Some of the sentences (like the first one shown above) make sense, while others (like the second one shown above) do not. The reason that these sentences do not make sense is that the relationship between the two nouns included in the sentence is not compatible with the pattern where the two nouns are inserted.
Sentences that make sense are labeled as `1` (*acceptable*), while sentences that do not make sense are labeled as `0` (*unacceptable*). The labeling is done by human annotators.

Participants in this shared task were asked to train a machine learning model to predict whether sentences are acceptable or unacceptable. The best model was then applied to a secret test set that was not provided to the participants, and the highest score won.

We have downloaded the data from this task from their [Github repository](https://github.com/shammur/SemEval2022Task3). In total, we have approximately 20000 sentences.

## Dependencies

The following modules and packages are needed to load the data and to manipulate them. 

**Unless otherwise stated**, however, **DO NOT** use any method from these modules for any ML-related goal. 

In [ ]:
import re
import random
import collections

import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer

%matplotlib inline  
plt.rcParams['figure.figsize'] = (15, 10)  

---

# Data Description

For this assignment we pre-processed the sentences by tokenizing the text and annotating the grammatical category and lemma of each token (more details below). These data are available in the `labeled_data_preprocessed.csv` file

In [ ]:
df = pd.read_csv("labeled_data_preprocessed.csv", index_col = 4)
annotation_re = re.compile(r"(.+)/(.+)")
df['pos_tagged_sentence'] = df['pos_tagged_sentence'].apply(lambda x: [annotation_re.findall(s)[0] for s in x.split()])
df['lemmatized_sentence'] = df['lemmatized_sentence'].apply(lambda x: [annotation_re.findall(s)[0] for s in x.split()])
df.sample(5)

## A note on Pandas

By executing the code above we've loaded our csv file as a Python Pandas DataFrame. If you are not familiar with this data structure you can easily cast our DataFrame into a  **dictionary of dictionaries**, as follows:

In [ ]:
df_dict = df.to_dict()

# note that the keys of the outmost dictionary are the column names
print(df_dict.keys())

# while the keys of each embedded dictionary are the rows indices
print(list(df_dict['raw_sentence'].keys())[:100])

## Columns keys

**`class`** = class label the sentence (**this is our target label**).

    0 - unacceptable
    1 - acceptable
   
**`raw_sentence`** = raw sentence text

**`pos_tagged_sentence`** = an annotated version of each sentence obtained by:

    - lowercasing the sentence
    - tokenizing the text (roughly, splitting the into morphological units such as words and punctuation marks)
    - annotating the grammatical category (a.k.a. Part of Speech: PoS) of each token

**`lemmatized_sentence`** = annotated version of each sentence obtained by using the annotation available in the `pos_tagged_sentence` representation in order to tag the base form (e.g. "*be*" for the inflected word "*am*") of each token.

## Tagging notation

The PoS-tagged and the lemmatized sentences have been represented using the [nltk notation](http://www.nltk.org/book_1ed/ch05.html), where 

- each **sentence** is represented as a list of annotated tokens

- each **annotated token** is represented as a `(word, tag)` tuple

For instance, we can verbalize this PoS annotation:

`[('The', 'DET'), ('grand', 'ADJ'), ('jury', 'NOUN'), ('commented', 'VERB')]`

as:

> *"the determiner "the" is followed by the adjective "grand", in turn followed by the noun "jury" and the verb "commented"*

... while the following lemmatized text 

`[('The', 'The'), ('grand', 'grand'), ('jury', 'jury'), ('commented', 'comment')]`

can be read as 

> *the text is composed by inflected forms of the lemmas "the", "grand", "jury" and "comment"* (note that the only token that is different from its base form is the last one)

### Part of Speech TagSet

For PoS-tagging our sentences we've used the Universal tagset proposed by [Petrov et al (2011)](https://arxiv.org/abs/1104.2086):

|  tag  | meaning |
|:-----:|:---------|
| VERB | verbs (all tenses and modes) |
| NOUN | nouns (common and proper) |
| PRON | pronouns |
| ADJ | adjectives |
| ADV | adverbs |
| ADP | adpositions (prepositions and postpositions) |
| CONJ | conjunctions |
| DET | determiners (e.g., articles) |
| NUM | cardinal numbers |
| PRT | particles or other function words |
| X | other: foreign words, typos, abbreviations |
| \. | punctuation |

## Distribution of the judgments

As shown by the following barplot, about half of the sentences belong to each class. In other words, we are going to work with a **balanced dataset**.

In [ ]:
df['class'].value_counts(sort = False).plot.bar(color='#1f77b4')

---

# Data Preparation

## Feature extraction

In this step, we'll extract the features that we will later used to convert each sentence into a vector. Let's store all the estimates as subdictionaries of a `features_dictionary` dictionary of dictionaries:

In [ ]:
# Keys are feature names, values are dictionaries mapping sentence indices to feature values
features_dictionary = dict() 

**1.** **Number of tokens in each sentence** (0.5 points)

Calculate the number of tokens (i.e. linguistic units) in each sentence. 

Store these numbers in a `features_dictionary` sub-dictionary labelled `tokens_counts`, whose keys are the `sentence_id` and the values are the number of tokens of each sentence. 

**Answer.**

In [ ]:
tokens_counts = dict()
sentence = df_dict['pos_tagged_sentence']

for i in range(len(list(sentence.items() ) ) ):
    tokens_counts[list(sentence.keys() )[i] ] = len(list(sentence.values() )[i])
    
features_dictionary['tokens_counts'] = tokens_counts

In [ ]:
def test_token_counts():
    key = 'tokens_counts'
    assert len(features_dictionary) == 1, 'features_dictionary should contain a single key "tokens_counts"'
    assert len(features_dictionary[key]) == len(df), f"{len(features_dictionary[key])} {key}, {len(df)} sentences"
    assert all([el >= 0 for el in features_dictionary[key].values()]), f'{key} should be non-negative'
    assert all([type(el) == int for el in features_dictionary[key].values()]), f'{key} should be integers'
    assert set(features_dictionary[key].keys()) == set(df.index.tolist()), f'{key} indices should match sentence ids'

test_token_counts()

**2.** **Number of characters in each sentence** (0.5 points)

Calculate the number of characters in each **lemmatized sentence**. 

Store this estimates in a `features_dictionary` sub-dictionary labelled `characters_counts`, whose keys are the `sentence_id` and the values are the number of characters of each sentence. 

**Answer.**

In [ ]:
characters_counts = dict()
sentence = df_dict['lemmatized_sentence']
for i in range(len(list(sentence.items() ) ) ):
    counter = 0
    for j in list(sentence.values() )[i]:
        lemmatized = j[1]
        counter += len(lemmatized)
    characters_counts[list(sentence.keys() )[i] ] = counter

features_dictionary['characters_counts'] = characters_counts

In [ ]:
def test_char_counts():
    key = 'characters_counts'
    assert len(features_dictionary) == 2, 'features_dictionary keys should be "tokens_counts" and "characters_counts"'
    assert len(features_dictionary[key]) == len(df), f"{len(features_dictionary[key])} {key}, {len(df)} sentences"
    assert all([el >= 0 for el in features_dictionary[key].values()]), f'{key} should be non-negative'
    assert all([type(el) == int for el in features_dictionary[key].values()]), f'{key} should be integers'
    assert set(features_dictionary[key].keys()) == set(df.index.tolist()), f'{key} indices should match sentence ids'

test_char_counts()

**3.** **n-grams features** (1 point)

As you have learned (or will shortly learn) in *Computationele Linguistiek*, a common way to enhance features is to use $n$-grams, i.e., a contiguous sequence of $n$ words. Write a piece of code that:

a) finds the 200 most frequent $n$-grams in our *lemmatized* sentences, with $n$ equal to 1, 2, or 3. Store these $n$-grams in a set called `most_frequent_ngrams`

b) counts how many times each of these `most_frequent_ngrams` appears in each sentence and stores these estimates in a dictionary of dictionaries called `ngrams_frequencies`, in which the keys are the $n$-grams (e.g. `my god`) and the values are dictionaries of the form `{sentence_id : occurrences_of_this_ngram}`. 

*Hint*: the `nltk.ngrams(sequence, n)` method can be used to generate all the possible length-$n$ $n$-grams from a sequence, as in the following examples (ignore eventual `DeprecationWarning` warnings):

In [ ]:
print(list(nltk.ngrams(["I", "came", "here", "to", "rescue", "you", "from", "him"], 1)))
print(list(nltk.ngrams(["I", "came", "here", "to", "rescue", "you", "from", "him"], 2)))
print(list(nltk.ngrams(["I", "came", "here", "to", "rescue", "you", "from", "him"], 3)))

**Answer.**

In [ ]:
sentence = df_dict['lemmatized_sentence']
all_ngrams = list()
ngrams_frequencies = dict()

for i in sentence.items():
    lemmatized_sentence = [x[1] for x in i[1]]
    for n in [1,2,3]:
        for n_gram in list(nltk.ngrams(lemmatized_sentence, n)):
            all_ngrams.append(n_gram)

most_frequent_ngrams = {x[0] for x in collections.Counter(all_ngrams).most_common(200)}         

for ngram in most_frequent_ngrams:
    occurrences = dict()
    for i in list(sentence.items()):
        count = 0
        lemmatized_sentence = [x[1] for x in i[1]]    
        sentence_ngram = list(nltk.ngrams(lemmatized_sentence, len(ngram)))
        for x in sentence_ngram:
            if x == ngram:
                count += 1
        occurrences[i[0]] = count
    ngrams_frequencies[ngram] = occurrences

## Reorganizing our features

Up to this moment, we have organized our features into the following dictionaries:

1. `ngrams_frequencies`: a group of lexical features, encoding how many times a given sequence of words (i.e. an n-gram) occurs in each sentence

1. `features_dictionary`: the numbers of tokens and characters in each sentence

In order to train our models, we're going to convert these dictionaries into matrices, where each row vector represent a single sentence. 

The following script converts the `features_dictionary` into a `other_features_matrix`, whose columns are the keys of `features_dictionary`, and the rows are the sentence ids. Similarly, it converts `ngrams_frequencies` into `ngrams_matrix`.

In [ ]:
def to_matrix(feature_dict: dict, sentence_ids: list) -> np.ndarray:
    # feature_dict is one of features_dictionary, ngrams_frequencies
    feature_names = feature_dict.keys()
    # let's create a matrix of zeros and populate it by iterating over our dictionary 
    feature_matrix = np.zeros((len(sentence_ids), len(feature_names)))
    for fix, feature_name in enumerate(feature_names):
        for tix, sentence_id in enumerate(sentence_ids):
            feature_matrix[tix, fix] = feature_dict[feature_name][sentence_id]
    return feature_matrix

In [ ]:
sentence_ids = sorted(df_dict['raw_sentence'].keys())
other_features_matrix = to_matrix(features_dictionary, sentence_ids)
ngrams_matrix = to_matrix(ngrams_frequencies, sentence_ids)

## Target variable

Next, we create an array with the class label for each sentence.

In [ ]:
sentence_classes = np.asarray([df_dict['class'][i] for i in sentence_ids])

## Weighting the lexical feature frequencies

Absolute frequencies may be a deceiving source of information: a word may appear frequently in a sentence simply due to its overall high frequency, while the use of an overall rare word may be an useful semantic or pragmatic clue. As a consequence, it is common practice to **weight** the frequency of a word so as to take into account its overall distribution (sometimes, along with the distribution of all the words in the corpus). 

Probably the most widely used statistic in information retrieval and text mining is the [term frequency-inverse document frequency (tf-idf)](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), in which the weigth of each term $t$ in each document $d$ is estimated as:

$$\text{tf-idf}_{t,d} = \text{tf}_{t,d} \times \text{idf}_t = \text{tf}_{t,d} \times log \left( \frac{N}{\text{df}_t}\right)$$

- where:
    - tf$_{t,d}$ is the number of occurrences of term $t$ in document $d$ (the term frequency)
    - $N$ is the total number of documents
    - df$_t$ is the number of documents in which term $t$ appears (the document frequency)
    
We transform `ngrams_matrix` into tf-idf representations called `ngrams_matrix_tfidf` using the `TfidfTransformer()` from sklearn (see the [docs](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) for more info).

In [ ]:
# let's weight our matrix (norm="l1" because we chose to adjust the term frequency for document length)
ngrams_matrix_tfidf = TfidfTransformer(norm="l1").fit_transform(ngrams_matrix).todense()

## Merging our feature matrices

**4. Merging our feature matrices** (0.5 points)

Create a feature matrix by merging the non-lexical features with the lexical feature set

- `features_set_ngrams`, obtained by merging `other_features_matrix` and `ngrams_matrix_tfidf`


*Hint*: the numpy.concatenate() method can be used to concatenate two matrices

**Answer.**

In [ ]:
features_set_ngrams = np.concatenate((other_features_matrix, ngrams_matrix_tfidf), axis = 1)        

## Dataset splitting

**5. Development and Test Set** (0.5 points)

As a first step, split the whole dataset into a development and a test set: randomly select 10% of the sentences to be used in the next phase of this activity (i.e. "Test the best performing model"). This is done mainly to prevent our models from over-fitting the training data.

The remaining 90% of our sentences will compose the development set on which we are going to train our model.

Divide the ids of our matrix into a test set (approximately 10% of the ids of the rows of our matrices) and a development set (the remaining rows)

**Answer.**

In [ ]:
length = len(features_set_ngrams)
index_testset = random.sample(range(length-1), round(0.1*length))
index_developmentset = list(range(length-1))
for i in index_developmentset:
    if i in index_testset:
        index_developmentset.remove(i)

---

# Logistic Regression

In the second step, we will see how a Machine Learning algorithm we discussed in class, Logistic Regression, performs with the dataset we've created in the previous phase. For implementing Logistic Regression, we'll use the `linear_model` module available in the `scikit-learn` library (**please note that you are not allowed to use any other module of this library**)

## Implementing our ML algorithm

The following example shows how you can use the `scikit-learn` library to implement Logistic Regression:

In [ ]:
# some dummy datasets

X_train = np.random.randint(3, size=(20, 6))  # dummy datatset of training data
X_test = np.random.randint(3, size=(5, 6))  # dummy datatset of test data

y_train = np.random.randint(3, size=20)  # dummy labels for the training data
y_test = np.random.randint(3, size=5)  # dummy labels for the test data

In [ ]:
# a minimal Logistic Regression

# Create logistic regression object
regr = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial')

#Train the model using the training sets
regr.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = regr.predict(X_test)

Having trained a model, you would then estimate its performance by comparing `y_pred` (the predicted labels) and `y_test` (the gold standard)

## Performance Measures

To evaluate the quality of a model, we generate predictions on a held-out set (for which we have the correct annotations) and we calculate:

**Accuracy**: the percentage of inputs documents (i.e., sentences) correctly classified

**Precision, Recall and F-measure** are calculated from the following estimates:

- True Positives: fraction of sentences that we correctly identified as relevant
- True Negatives: fraction of sentences that we correctly identified as irrelevant
- False Positives (or Type I errors): irrelevant sentences that we incorrectly identified as relevant
- False Negatives (or Type II errors): relevant sentences that we incorrectly identified as irrelevant

<img src="https://www.nltk.org/images/precision-recall.png" alt="drawing" width="600"/>

- **Precision**: how many of the items that we identified as relevant were actually relevant: $\frac{\text{TP}}{(\text{TP}+\text{FP})}$.


- **Recall**: how many of the relevant items we identified we identified as relevant, is $\frac{\text{TP}}{(\text{TP}+\text{FN})}$,


- **F$_1$-measure**: the harmonic mean of the precision and recall: $2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

## Cross-validation

Below we will train our logistic regression model with two feature sets:

- `other_features_matrix` alone

- `features_set_ngrams`, which includes `other_features_matrix` as well as our $n$-grams features

When comparing different models, it is better to perform multiple evaluations on different portions of our develpment set, and then combining the scores from those evaluations. This techinique is called **cross validation**.

Now, we will divide our development set into **5 subsets** called folds. For each fold, we train our models using all the data in the develpment set except those in this fold, and then we test our models on this fold. We then combine our evaluation to compute the overal score of our models. 

### Using NumPy

Some of the computations you will perform below, such as creating a training set from the development set, will require selecting some elements from a NumPy array or matrix. We actively encourage you to perform such operations using matrices entirely, i.e., using NumPy methods such as `numpy.take`, rather than to create `for` loops. Practice with NumPy will be essential in your follow-up course *Machine Learning*.

**6. Logistic regression training and cross validation** (2 points). Follows these steps to compare our settings:

1. Shuffle development dataset (you may consider using the `random.shuffle()` method)

2. Split the shuffled development set into 5 groups (you may consider using the `np.array_split()` method)

3. For each fold:
    - take the fold as the held-out data set and the remaining folds together as a training set
    - train Logistic Regression with each feature setting (lemmas vs. ngrams) on the training set
    - evaluate each ML algorithm + feature setting on the held-out set
    - for each model (manually, i.e., without using existing libraries) calculate accuracy, precision, recall and f-measure and store these results
    
4. for each feature setting, average the performance measures over all the folds in cross-validation (this is called **macro-averaging**)

5. choose the best performing feature setting

**Answer.**

In [ ]:
random.shuffle(index_developmentset)
index_folded_developmentset = np.array_split(index_developmentset, 5)
performance = []

for set in range(2) :
    if set == 0 :
        current_featureset = features_set_ngrams
    else :
        current_featureset = other_features_matrix
       
    performance_feature = []
    
    for fold in index_folded_developmentset :
        # get all the indexes of the remaining folds
        index_trainset = index_developmentset.copy()
        for i in index_trainset:
            if i in fold:
                index_trainset.remove(i)
        
        # Create logistic regression object
        regr = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=500)
        
        # Train the model using the training sets
        regr.fit(current_featureset[index_trainset,:], sentence_classes[index_trainset]) 
        
        # Predict the response for test dataset
        y_pred = regr.predict(current_featureset[fold,:])

        # Get the gold standard labels 
        y_gold = sentence_classes[fold]
        
        # set values important for recall precision and f measure
        true_positive = 0
        true_negative = 0
        false_positive = 0
        false_negative = 0
        for i in range(len(y_pred)) :
            if y_gold[i] == 1 and y_pred[i] == 1 :
                true_positive += 1
            elif y_gold[i] == 0 and y_pred[i] == 0 :
                true_negative += 1
            elif y_gold[i] == 1 and y_pred[i] == 0 :
                false_negative += 1
            elif y_gold[i] == 0 and y_pred[i] == 1 :
                false_positive += 1
        
        # calculate performance for this cross validation
        accuracy = (true_positive+true_negative)/(true_positive+false_negative+false_positive+true_negative)
        precision = true_positive / (true_positive + false_positive)
        recall = true_positive / (true_positive + false_negative)
        f_measure = 2 * ((precision * recall) / (precision + recall))
        performance_feature.append([accuracy, precision, recall, f_measure])
    
    med_accuracy = 0
    med_precision = 0
    med_recall = 0
    med_f_measure = 0
    for i in performance_feature :
        med_accuracy += i[0]
        med_precision += i[1]
        med_recall += i[2]
        med_f_measure += i[3]
    med_accuracy = med_accuracy / len(performance_feature)
    med_precision = med_precision / len(performance_feature) 
    med_recall = med_recall / len(performance_feature)
    med_f_measure = med_f_measure / len(performance_feature) 
        
    performance.append([med_accuracy, med_precision, med_recall, med_f_measure])
    
print("the order of the performance is: [accuracy, precision, recall, f_measure]")
print("performance featureset n-grams:    ", performance[0])
print("performance other features matrix: ", performance[1])

## Model evaluation on a test set

**7. Test the best performing model** (1 point)

Assess the performance of the better model identified in the previous step. For this evaluation, we will use as training set the full development set that we've used from the 5-folds cross-validation, corresponding to 90% of our sentences). We will then evaluate our model on the remaining 10% of sentences that we've never used (up to now), i.e. the `test set`.

We will compare the prediction of our model against the human annotations and we will **manually** calculate:

- its accuracy

- its precision, recall and f-measure

- the confusion matrix (fell free to use the `matplotlib.pyplot.imshow()` method to plot it, but bear in mind that the confusion matrix should be calculated manually)

**Answer.**

In [ ]:
from sklearn.metrics import confusion_matrix
# Create logistic regression object
regr = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=500)
        
# Train the model using the entire development sets
regr.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset]) 
        
# Predict the response for test dataset
y_pred = regr.predict(features_set_ngrams[index_testset,:])

# Get the gold standard labels 
y_gold = sentence_classes[index_testset]

true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0
for i in range(len(y_pred)) :
    if y_gold[i] == 1 and y_pred[i] == 1 :
        true_positive += 1
    elif y_gold[i] == 0 and y_pred[i] == 0 :
        true_negative += 1
    elif y_gold[i] == 1 and y_pred[i] == 0 :
        false_negative += 1
    elif y_gold[i] == 0 and y_pred[i] == 1 :
        false_positive += 1
        
# calculate performance for this cross validation
accuracy = (true_positive+true_negative)/(true_positive+false_negative+false_positive+true_negative)
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
f_measure = 2 * ((precision * recall) / (precision + recall))

print("accuracy : ",accuracy,"\nprecision: ",precision,"\nrecall   : ",recall,"\nf_measure: ",f_measure)

confusion_matrix = np.array([[true_positive, false_positive], [false_negative, true_negative]])
alpha = ['Positive', 'Negative']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion_matrix)
fig.colorbar(cax)

ax.set_xticklabels(['']+alpha)
ax.set_yticklabels(['']+alpha)
plt.xlabel("The Gold Standard")
ax.xaxis.set_label_position('top') 
plt.ylabel("Predicted")
plt.show()

---

# Neural Networks

Now instead of logistic regression we will experiment with neural networks for this classification task. For this, you will need to install tensorflow (e.g. via Anaconda) if you haven't already.

In [ ]:
import tensorflow as tf

The goal of this exercise is not to find a classifier with the best out-of-sample performance, but rather to see the practical impact of various choices in setting up a neural network (some of which you have seen in the lecture, some not).

We will use Keras, which is a higher-level interface that can be used on top of tensorflow. It makes many common deep learning tasks very easy to perform. The following code sets up a simple neural network with one hidden layer of 20 neurons. We will use the Stochastic Gradient Descent (SGD) optimizer; see documentation here: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(2, activation='softmax')
])
sgd = tf.keras.optimizers.SGD()
model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

The hidden layer uses `tanh` as its nonlinear activation function. The output layer uses `softmax`, which is also a smooth nonlinear function, but has the special property that it transforms an arbitrary real-valued vector into a vector of numbers between 0 and 1, summing to 1. These values are interpreted as likelihoods for the classes, just as for logistic regression. The network is trained to minimize the loss function `sparse_categorical_crossentropy`, which corresponds to maximizing that likelihood, again like logistic regression.

**Note:** A loss function `categorical_crossentropy` also exists in Keras. If the labels were encoded as "one-hot" vectors instead of as integers, `categorical_crossentropy` would be the right loss function to use instead of the `sparse_` one.

**8. Train a simple neural network** Using `features_set_ngrams`, fit this model to your development data (with `epochs=30`; in each "epoch", all data points are passed through the network forward and backward once). Then evaluate the model on the test data (1 points)

**Answer.**

In [ ]:
model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=30)
model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

**9. Neural network parameter exploration** (2 points)

The `model.fit` function returns a History object, which stores among other things the training (in-sample) losses per epoch. If the History object is called `hist[i]`, the losses are `hist[i].history['loss']`.

Use this to plot the training losses per epoch for three different values of the `learning_rate` (the step size $\eta$, or "eta", used by stochastic gradient descent): 0.1, 0.01 (the default), and 0.001. (To write your solution, start by copy-pasting the code from above and putting a for-loop around it.)
What do you observe?

Similarly experiment with each of the following aspects of the neural network:
* You can pass different values of `batch_size` to `model.fit`. For `batch_size=1`, we get standard Stochastic Gradient Descent (SGD) where the gradient is computed for one data point at a time. For a batch size at least as large as the training data, the gradient is computed for all these data points together (this is batch GD). The default value of 32 is in between these two extremes (this is called a "minibatch");
* Gradient descent only looks at the gradient, and only at the point of the current weights. More advanced optimizers take more information into account, such as second derivatives and recently visited points. A popular choice is `optimizer='adam'` instead of `optimizer=sgd`;
* The number of epochs used when fitting the model (if the 30 epochs we chose before led to worse performance than logistic regression, try increasing the number of epochs; see if the accuracy seems to be converging);
* The activation function in the hidden layer (e.g. `'relu'` instead of `'tanh'`);
* The number of neurons in the hidden layer.

For each of these, write code that tries out models with different values of that setting, while keeping all other settings constant at values you choose based on the previous experiments. Your code must produce relevant output when the notebook is run. This can be one of many different things, e.g. training losses per epoch, test accuracy, computation time, ..., so for each setting, pick one that you expect to be interesting, and write code to measure it. In the corresponding text cell, write down your conclusions from your code's output.

**Answer.**

In [ ]:
for i in [0.1, 0.01, 0.001] :
    print("\n")
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    sgd = tf.keras.optimizers.SGD(learning_rate=i)
    model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
    hist = model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=30,)
    plt.plot(hist.history["loss"])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.show()
    model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

Learning rate: The model performs best with a learning rate of 0.01, the model then has a loss of 0.6174 and an accuracy of 0.7000

In [ ]:
for i in [1, 32, 20000] : 
    print("\n")
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    sgd = tf.keras.optimizers.SGD(learning_rate = 0.01)
    model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
    hist = model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=10, batch_size=i)
    plt.plot(hist.history["loss"])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.show()
    model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

Batch size: the model with a batch size of 32 performed the best, with a low loss and relatively high accuracy. Batch size 1 was excruciatingly slow and performed worst. A really big batch size (e.g. 20000) has more loss but a way better accuracy than a low batchsize.

In [ ]:
for i in range(2) : 
    print("\n")
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    if i == 0 :
        optimiser = tf.keras.optimizers.SGD(learning_rate = 0.01)
    else :
        optimiser = tf.keras.optimizers.Adam(learning_rate = 0.01)
        
    model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
    hist = model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=30)
    plt.plot(hist.history["loss"])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.show()
    model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

The adam optimizer is much better at optimizing loss and accuracy both. On the train set this seems to generalise to the test set

In [ ]:
for i in [15, 30, 45] : 
    print("\n")
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='tanh'),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    adam = tf.keras.optimizers.Adam(learning_rate = 0.01)
        
    model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
    hist = model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=i)
    plt.plot(hist.history["loss"])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.show()
    model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

The model performed best with 45 epochs in the trainset tho the generalisation to the test set seems to be better for 30 epochs (due to overfitting probably) 15 epochs seems to be too little epochs altough it is twice as fast as 30 while not performing that much worse. So this might be an option for quickly testing other hyper parameters. 

In [ ]:
for i in ["tanh", "relu"] : 
    print("\n")
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation=i),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    adam = tf.keras.optimizers.Adam(learning_rate = 0.01)
        
    model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', ])
              
    hist = model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=45)
    plt.plot(hist.history["loss"])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.show()
    model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

Between tanh and relu as activation function in the hidden layer, tanh and relu have almost the same accuracy. Overall relu has less loss than tanh, which gives it a better performance.

In [ ]:
for i in [10, 20, 30, 40] : 
    print("\n")
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(i, activation="relu"),
    tf.keras.layers.Dense(2, activation='softmax')
    ])
    adam = tf.keras.optimizers.Adam(learning_rate = 0.01)
        
    model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
    hist = model.fit(features_set_ngrams[index_developmentset,:], sentence_classes[index_developmentset], epochs=45)
    plt.plot(hist.history["loss"])
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.show()
    model.evaluate(features_set_ngrams[index_testset,:], sentence_classes[index_testset])

The model performed the best with 20 neurons in the hidden layer, a loss of 0.2359 and accuracy 0.8985

---

# Questions

**10.** What was your better logistic regression model? Intuitively, why do you think that it performed better? (0.5 points)

**Answer**

Our better logistic regression model is the "featureset n-grams", because the accuracy, precision, recall and f-measure of this model all scored higher than with the "other features" model. This big difference in performance can be explained by the fact that the "featureset n-grams" model got more features than the "other features" model. The extra n-gram feature that this model has makes the model perform really well.

**11.** Did the neural networks perform better or worse than logistic regression? How can you interpret that result? (0.5 points)

**Answer**

Initially, looking at the accuracy, the neural networks performed way worse than logistic regression, but with the right settings the neural networks performed a slightly higher accuracy than the logistic regression. This means that they are both a good method, but only when given the right parameters the neural networks outperform the logistic regression. So logistic regression is faster and easier to use but less accurate and neural networks are more work to find the right parameters but they are more accurate.